In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

# Load dataset
path_to_file = tf.keras.utils.get_file("shakespeare.txt", "input.txt")
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Preprocess data
chars = sorted(set(text))
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = np.array(chars)

# Prepare training data
seq_length = 100
step = 1  # Step size for sliding window
sequences = []
next_chars = []

# Create sequences more efficiently
print("Creating sequences...")
for i in range(0, len(text) - seq_length, step):
    sequences.append(text[i:i + seq_length])
    next_chars.append(text[i + seq_length])

print(f"Number of sequences: {len(sequences)}")

# Convert to numerical form
X = np.zeros((len(sequences), seq_length-1), dtype=np.int32)
y = np.zeros(len(sequences), dtype=np.int32)

print("Converting to numerical form...")
for i, sequence in enumerate(sequences):
    # Convert input sequence
    for t, char in enumerate(sequence[:-1]):  # All but last char
        X[i, t] = char2idx[char]
    # Convert target char
    y[i] = char2idx[next_chars[i]]

# Convert y to one-hot encoding
y = to_categorical(y, num_classes=len(chars))

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"Vocabulary size: {len(chars)}")


2025-07-12 00:38:30.023678: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-12 00:38:30.033469: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-12 00:38:30.388259: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-12 00:38:31.893403: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

Creating sequences...
Number of sequences: 1115294
Converting to numerical form...
X shape: (1115294, 99)
y shape: (1115294, 65)
Vocabulary size: 65


In [2]:
# Build LSTM model
model = tf.keras.Sequential([
tf.keras.layers.Embedding(len(chars), 64, input_length=seq_length-1),
tf.keras.layers.LSTM(256, return_sequences=True),

tf.keras.layers.LSTM(256),
tf.keras.layers.Dense(len(chars), activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=1, batch_size=128)


/home/danish/Projects/projects/deeplearning/.venv/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
E0000 00:00:1752260931.312888   11210 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1752260931.318455   11210 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


8714/8714 ━━━━━━━━━━━━━━━━━━━━ 2258s 259ms/step - accuracy: 0.3179 - loss: 2.4313
